Problem Set 2
Maire Taegan Mullane
4/13/25

In [2]:
import pandas as pd
import numpy as np
import math
import scipy as sp
import matplotlib.pyplot as plt
from itertools import combinations

In [3]:
file1_path = "census.mat"
mat_data = sp.io.loadmat(file1_path)

In [4]:
# Extract the variables
educ = mat_data['educ'].flatten()
y = mat_data['y'].flatten()
yob = mat_data['yob'].flatten()

# Create a DataFrame
df = pd.DataFrame({
    'educ': educ,
    'y': y,
    'yob': yob
})
df

,educ,y,yob
0,0,5.790019,30
1,0,5.952494,30
2,0,5.315949,30
3,0,5.595926,30
4,0,6.068915,30
...,...,...,...
329504,1,6.323980,39
329505,1,5.847161,39
329506,0,5.909597,39
329507,0,6.047781,39


1.a.

In [5]:
Y_1 = df[df.educ== 1]['y'].mean()
Y_0 = df[df.educ== 0]['y'].mean()

diff = (Y_1- Y_0)
print(f"Difference in means is: {diff:.2f}")


Difference in means is: 0.38


In [9]:
N = float(len(df))
M = float(len(df[df.educ == 1]))

# Calculate the variance of the difference in means
se_0 =(1/(N-M-1)) * sum((df[df.educ == 0].y - Y_0)**2)
se_1 = (1/(M - 1)) * sum((df[df.educ == 1].y - Y_1)**2)
var = math.sqrt(se_0/(N-M) + se_1/M)
print(f"Variance of the difference in means is: {var:.8f}")

Variance of the difference in means is: 0.00231002


1.b.

In [17]:
def bootstrap_diff(df, n=None):
    df_b = df.sample(n, replace=True)
    
    Y_1_b = df_b[df_b.educ == 1]['y'].mean()
    Y_0_b = df_b[df_b.educ == 0]['y'].mean()
    return Y_1_b - Y_0_b

def bootstrap_se(df, B=1000):
    n = len(df)
    diffs = np.zeros(B)
    for i in range(B):
        diffs[i] = bootstrap_diff(df, n)
    return np.std(diffs)

# Set the number of bootstrap samples
B = 1000
# Calculate the bootstrap standard error
bootstrap_se_value = bootstrap_se(df, B=B)
print(f"Bootstrap standard error of the difference in means is: {bootstrap_se_value:.8f}")


Bootstrap standard error of the difference in means is: 0.00226623


1.c.

In [47]:
yob_values = df['yob'].unique()

In [23]:
df['const'] = 1
X = df[["const", "educ"]].to_numpy()
Y = df[["y"]].to_numpy()
beta = np.linalg.inv(X.transpose()@X) @ X.transpose()@Y
errors = Y - X @ beta
# Calculate the inverse matrix once and store it
X_T_X_inv = np.linalg.inv(X.T @ X)

In [26]:
middle_cluster = np.zeros([X.shape[1], X.shape[1]])

for cluster in df["yob"].unique():
    
    X_g = X[df["yob"] == cluster,:]
    errors_g = errors[df["yob"] == cluster,:]
    middle_cluster += np.sum(X_g * errors_g, axis = 0).T @ np.sum(X_g * errors_g, axis = 0)
    
V_LZ = X_T_X_inv @ middle_cluster @ X_T_X_inv
V_LZ[1,1]**0.5

np.float64(0.008415067653131372)

1.d.

In [28]:
w_k = np.zeros(len(yob_values))
for i in range(len(yob_values)):
    w_k[i] = np.mean(df[df['yob']==yob_values[i]].educ)
w_k

array([0.36417475, 0.38279436, 0.38831455, 0.39406849, 0.39660358,
       0.40316724, 0.40537397, 0.40713592, 0.41498453, 0.41974585])

In [49]:
def cluster_diff(df, i=None):
    df_c = df[df.yob == i].copy()
    # df_c_1 = df_c[df_c.educ == 1]
    # df_c_0 = df_c[df_c.educ == 0]

    w_kg = np.random.choice(w_k, size=1, replace=True)[0]
    n_c = len(df_c)
    n_treat = int(n_c * w_kg)
    # df_cb_1 = df_c_1.sample(int(n_c*w_kg), replace=True)
    # df_cb_0 = df_c_0.sample(int(n_c*(1-w_kg)), replace=True)

    # df_cb = pd.concat([df_cb_1, df_cb_0])

    df_c['educ'] = 0
    treated_indices = np.random.choice(df_c.index, size=n_treat, replace=False)
    df_c.loc[treated_indices, 'educ'] = 1

    return df_c

def cluster_se(df, cluster = None):
    full_sample = pd.DataFrame()

    for i in range(len(cluster)):
        df_i = cluster_diff(df, cluster[i])
        full_sample = pd.concat([full_sample, df_i], ignore_index=True)

    full_sample['const'] = 1
    X = full_sample[["const", "educ"]].to_numpy()
    Y = full_sample[["y"]].to_numpy()
    beta = np.linalg.inv(X.transpose()@X) @ X.transpose()@Y
        
    return beta[1][0]

def bootstrap_cluster_se(df, B=1000):
    diffs = np.zeros(B)
    for i in range(B):
        diffs[i] = cluster_se(df, yob_values)
    return np.std(diffs)

# Set the number of bootstrap samples
B = 1000
# Calculate the bootstrap standard error
bootstrap_se_value = bootstrap_cluster_se(df, B=B)

print(f"Cluster bootstrap standard error of the difference in means is: {bootstrap_se_value:.8f}")


Cluster bootstrap standard error of the difference in means is: 0.00249042


In [ ]:
old version
# def cluster_diff(df, cluster_id, w_k):
#     df_c = df[df.yob == cluster_id]
#     n_c = len(df_c)
#     w_kg = np.random.choice(w_k, size=1)[0]

#     df_treated = df_c[df_c.educ == 1]
#     df_control = df_c[df_c.educ == 0]

#     n_treat = int(n_c * w_kg)
#     n_control = n_c - n_treat

#     sample_treat = df_treated.sample(n=n_treat, replace=True)
#     sample_control = df_control.sample(n=n_control, replace=True)

#     return pd.concat([sample_treat, sample_control])

# def hybrid_cluster_bootstrap(df, B=1000, w_k=np.linspace(0.1, 0.9, 9)):
#     clusters = df['yob'].unique()
#     boot_coefs = []

#     for _ in range(B):
#         sampled_clusters = np.random.choice(clusters, size=len(clusters), replace=True)
#         boot_sample = pd.DataFrame()

#         for cluster_id in sampled_clusters:
#             cluster_data = cluster_diff(df, cluster_id, w_k)
#             boot_sample = pd.concat([boot_sample, cluster_data])

#         boot_sample['const'] = 1
#         X = boot_sample[['const', 'educ']].to_numpy()
#         Y = boot_sample[['y']].to_numpy()

#         beta = np.linalg.inv(X.T @ X) @ X.T @ Y
#         boot_coefs.append(beta[1, 0])  # Coefficient on educ

#     return np.std(boot_coefs)


# se = hybrid_cluster_bootstrap(df, B=B, w_k=w_k)
# print(f"Cluster bootstrap SE with treatment resampling: {se:.8f}")

1.e.

In [40]:
df2 = df.copy()
# Group-mean center y (within-cluster)
df2['y_group_mean'] = df2.groupby('yob')['y'].transform('mean')
df2['y_within'] = df2['y'] - df2['y_group_mean']

# Total variance
var_total = df2['y'].var(ddof=0)

# Within-cluster variance
var_within = df2['y_within'].var(ddof=0)

# Ratio = share of variance that is within clusters (i.e., 1 - ICC)
within_cluster_share = var_within / var_total
print("Within-cluster share of variance (Y):", within_cluster_share)

# Calculate the ICC for y   
icc = 1 - (var_within / var_total)
print("ICC (between-cluster correlation):", icc)


Within-cluster share of variance (Y): 0.9999700951390741
ICC (between-cluster correlation): 2.990486092591116e-05


In [46]:
# ICC for log earnings (y)
model_y = mixedlm("y ~ 1", df, groups=df["yob"])
result_y = model_y.fit()

var_between_y = result_y.cov_re.iloc[0, 0]
var_within_y = result_y.scale
icc_y = var_between_y / (var_between_y + var_within_y)
print(f"Random-effects ICC for y (log-earnings): {icc_y:.4f}")


/Users/tmullane/Library/Python/3.9/lib/python/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/Users/tmullane/Library/Python/3.9/lib/python/site-packages/statsmodels/regression/mixed_linear_model.py:2200: ConvergenceWarning: Retrying MixedLM optimization with lbfgs
  warnings.warn(
/Users/tmullane/Library/Python/3.9/lib/python/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/Users/tmullane/Library/Python/3.9/lib/python/site-packages/statsmodels/regression/mixed_linear_model.py:2200: ConvergenceWarning: Retrying MixedLM optimization with cg
  warnings.warn(


Random-effects ICC for y (log-earnings): 0.0008


/Users/tmullane/Library/Python/3.9/lib/python/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/Users/tmullane/Library/Python/3.9/lib/python/site-packages/statsmodels/regression/mixed_linear_model.py:2206: ConvergenceWarning: MixedLM optimization failed, trying a different optimizer may help.
  warnings.warn(msg, ConvergenceWarning)
/Users/tmullane/Library/Python/3.9/lib/python/site-packages/statsmodels/regression/mixed_linear_model.py:2218: ConvergenceWarning: Gradient optimization failed, |grad| = 297.089556
  warnings.warn(msg, ConvergenceWarning)
/Users/tmullane/Library/Python/3.9/lib/python/site-packages/statsmodels/regression/mixed_linear_model.py:2237: ConvergenceWarning: The MLE may be on the boundary of the parameter space.
  warnings.warn(msg, ConvergenceWarning)
/Users/tmullane/Library/Python/3.9/lib/python/site-packages/statsmode

1.f.

In [45]:
# ICC for education (educ)
model_educ = mixedlm("educ ~ 1", df, groups=df["yob"])
result_educ = model_educ.fit()

var_between_educ = result_educ.cov_re.iloc[0, 0]
var_within_educ = result_educ.scale
icc_educ = var_between_educ / (var_between_educ + var_within_educ)
print(f"Random-effects ICC for educ: {icc_educ:.4f}")

Random-effects ICC for educ: 0.0011


/Users/tmullane/Library/Python/3.9/lib/python/site-packages/statsmodels/regression/mixed_linear_model.py:2237: ConvergenceWarning: The MLE may be on the boundary of the parameter space.
  warnings.warn(msg, ConvergenceWarning)


In [42]:
df2 = df.copy()
# Group-mean center y (within-cluster)
df2['educ_group_mean'] = df2.groupby('yob')['educ'].transform('mean')
df2['educ_within'] = df2['educ'] - df2['educ_group_mean']

# Total variance
var_total = df2['educ'].var(ddof=0)

# Within-cluster variance
var_within = df2['educ_within'].var(ddof=0)

# Ratio = share of variance that is within clusters (i.e., 1 - ICC)
within_cluster_share = var_within / var_total
print("Within-cluster share of variance educ:", within_cluster_share)

# Calculate the ICC for educ   
icc = 1 - (var_within / var_total)
print("ICC (between-cluster correlation):", icc)


Within-cluster share of variance educ: 0.9989657813580346
ICC (between-cluster correlation): 0.001034218641965401


1.g. We could also report the two stage clustered bootstrapped standard error.

2.a. I'd report EHW. 

In [53]:
fract = df.groupby("yob")["educ"].mean()
fract

yob
30    0.364175
31    0.382794
32    0.388315
33    0.394068
34    0.396604
35    0.403167
36    0.405374
37    0.407136
38    0.414985
39    0.419746
Name: educ, dtype: float64

In [54]:
fract.mean()

np.float64(0.3976363242160999)

In [55]:
fract.mean() * (1-fract.mean()) ** 0.5

np.float64(0.30861386806538577)

In [56]:
fract.var()

np.float64(0.0002679053097953354)